In [0]:

spark.conf.set("fs.azure.account.auth.type.mystorageaccountrachit.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.mystorageaccountrachit.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.mystorageaccountrachit.dfs.core.windows.net", "d569b312-0839-4220-a432-8f2bc9a03642")
#spark.conf.set("fs.azure.account.oauth2.client.secret.mystorageaccountrachit.dfs.core.windows.net", "Jx-8Q~Olxb.T85uT7-JBldgLN1KeLYmpasVv4b5T")
spark.conf.set("fs.azure.account.oauth2.client.secret.mystorageaccountrachit.dfs.core.windows.net", dbutils.secrets.get(scope='rachitscope', key='app-secret'))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.mystorageaccountrachit.dfs.core.windows.net", "https://login.microsoftonline.com/9958db81-6cef-48c5-b994-0f6bbc11e783/oauth2/token")

In [0]:
%run "/Workspace/DatabricksWorkspaceV1/DatabricksMasterclass"

## Delta Lake

In [0]:
df_sales.write.format('delta')\
                .mode('overwrite')\
                .save('abfss://destination@mystorageaccountrachit.dfs.core.windows.net/sales')

## Managed vs External Delta Tables

##### Database

In [0]:
%sql
CREATE DATABASE IF NOT EXISTS hive_metastore.salesDB;
USE CATALOG hive_metastore;

#### Managed table

In [0]:
%sql
CREATE TABLE salesDB.man_table 
(id INT,
 name STRING,
 marks int)
 USING DELTA;

In [0]:
%sql
INSERT INTO salesdb.man_table VALUES(1,'Rachit',30),(2,'Robert',25),(3,'Ansh',35),(4,'Donald',40);

In [0]:
%sql
SELECT * FROM salesdb.man_table;

In [0]:
%sql
DROP TABLE salesdb.man_table;
SELECT * FROM salesdb.man_table;


### External Table

In [0]:
%sql
--DROP TABLE salesdb.ext_table;
CREATE TABLE salesDB.ext_table 
(
  id INT,
 name STRING,
 marks int
 )
 USING DELTA 
 LOCATION 'abfss://destination@mystorageaccountrachit.dfs.core.windows.net/salesDB/exttable'

In [0]:
%sql
INSERT INTO salesdb.ext_table VALUES(1,'Rachit',30),(2,'Robert',25),(3,'Ansh',35),(4,'Donald',40);

In [0]:
%sql
SELECT * FROM salesdb.ext_table;

###Delta Table Functionalities

##### INSERT

In [0]:
%sql
INSERT INTO salesdb.ext_table VALUES(5,'Gulyani',30),(6,'Farcas',25),(7,'Lamba',35),(8,'Trump',40);

##### DELETE

In [0]:
%sql
DELETE FROM salesDB.ext_table
WHERE id = 8;


### DATA VERSIONING

In [0]:
%sql
DESCRIBE HISTORY salesDB.ext_table;

In [0]:
%sql
SELECT * FROM salesDB.ext_table
VERSION AS OF 2
ORDER BY ID;


#### TIME TRAVEL

In [0]:
%sql
RESTORE TABLE salesDB.ext_table TO VERSION AS OF 2;

In [0]:
%sql
SELECT * FROM salesdb.ext_table;

## VACUUM

In [0]:
%sql
VACUUM salesdb.ext_table RETAIN 2 HOURS; --Default retention time=> 7 days

In [0]:
%sql
--VACUUM salesdb.ext_table RETAIN 0 HOURS; --Default retention time=> 7 days

### **DELTA TABLE OPTIMISATION**

#### OPTIMIZE

In [0]:
%sql
OPTIMIZE salesdb.ext_table;

In [0]:
%sql
SELECT * FROM salesdb.ext_table;

#### ZORDER BY

In [0]:
%sql
OPTIMIZE salesdb.ext_table ZORDER BY (id);

In [0]:
%sql
SELECT * FROM salesDB.ext_table;

### AUTO LOADER

### Streaming dataframe

In [0]:
df= spark.readStream.format('cloudFiles')\
                    .option('cloudFiles.format', 'parquet')\
                    .option('cloudFiles.schemaLocation','abfss://aldestination@mystorageaccountrachit.dfs.core.windows.net/checkpoint')\
                    .load('abfss://alsource@mystorageaccountrachit.dfs.core.windows.net')

In [0]:
df.writeStream.format('delta')\
              .option('checkpointLocation', 'abfss://aldestination@mystorageaccountrachit.dfs.core.windows.net/checkpoint')\
              .trigger(processingTime='5 seconds')\
              .start('abfss://aldestination@mystorageaccountrachit.dfs.core.windows.net/data')